## Tabular Income Classifier Production Deployment

To run this notebook you need the inference data. This can be acquired in two ways:

  * Run train.ipynb
  * `gsutil cp -R gs://seldon-models/scv2/examples/income/infer-data .`

In [1]:
import numpy as np
import json
import requests

In [24]:
with open('./infer-data/test.npy', 'rb') as f:
    x_ref = np.load(f)
    x_h1 = np.load(f)
    y_ref = np.load(f)
    x_outlier = np.load(f)

In [3]:
reqJson = json.loads('{"inputs":[{"name":"input_1","data":[],"datatype":"FP32","shape":[]}]}')
url = "http://0.0.0.0:9000/v2/models/model/infer"

In [4]:
def infer(resourceName: str, batchSz: int, requestType: str):
    if requestType == "outlier":
        rows = x_outlier[0:0+batchSz]
    elif requestType == "drift":
        rows = x_h1[0:0+batchSz]
    else:
        rows = x_ref[0:0+batchSz]
    reqJson["inputs"][0]["data"] = rows.flatten().tolist()
    reqJson["inputs"][0]["shape"] = [batchSz, rows.shape[1]]
    headers = {"Content-Type": "application/json", "seldon-model":resourceName}
    response_raw = requests.post(url, json=reqJson, headers=headers)
    print(response_raw)
    print(response_raw.json())

### Pipeline with model, drift detector and outlier detector

In [5]:
!cat ../../models/income-preprocess.yaml
!echo "---"
!cat ../../models/income.yaml
!echo "---"
!cat ../../models/income-drift.yaml
!echo "---"
!cat ../../models/income-outlier.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: income-preprocess
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/scv2/examples/income/preprocessor"
  requirements:
  - sklearn
---
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: income
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/scv2/examples/income/classifier"
  requirements:
  - sklearn
---
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: income-drift
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/scv2/examples/income/drift-detector"
  requirements:
    - mlserver
    - alibi-detect
---
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: income-outlier
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/scv2/examples/income/outlier-detector"
  requirements:
    - mlserver
    - alibi-detect


In [6]:
!seldon model load -f ../../models/income-preprocess.yaml
!seldon model load -f ../../models/income.yaml
!seldon model load -f ../../models/income-drift.yaml
!seldon model load -f ../../models/income-outlier.yaml

{}
{}
{}
{}


In [7]:
!seldon model status income-preprocess -w ModelAvailable | jq .
!seldon model status income -w ModelAvailable | jq .
!seldon model status income-drift -w ModelAvailable | jq .
!seldon model status income-outlier -w ModelAvailable | jq .

{}
{}
{}
{}


In [8]:
!cat ../../pipelines/income.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: income-production
  namespace: seldon-mesh
spec:
  steps:
    - name: income
    - name: income-preprocess
    - name: income-outlier
      inputs:
      - income-preprocess
    - name: income-drift
      batch:
        size: 20
  output:
    steps:
    - income
    - income-outlier.outputs.is_outlier


In [9]:
!seldon pipeline load -f ../../pipelines/income.yaml

{}


In [10]:
!seldon pipeline status income-production -w PipelineReady| jq -M .

{
  "pipelineName": "income-production",
  "versions": [
    {
      "pipeline": {
        "name": "income-production",
        "uid": "cbimdu958n94aofi9tjg",
        "version": 1,
        "steps": [
          {
            "name": "income"
          },
          {
            "name": "income-drift",
            "batch": {
              "size": 20
            }
          },
          {
            "name": "income-outlier",
            "inputs": [
              "income-preprocess.outputs"
            ]
          },
          {
            "name": "income-preprocess"
          }
        ],
        "output": {
          "steps": [
            "income.outputs",
            "income-outlier.outputs.is_outlier"
          ]
        },
        "kubernetesMeta": {
          "namespace": "seldon-mesh"
        }
      },
      "state": {
        "pipelineVersion": 1,
        "status": "PipelineReady",
        "reason": "Created pipeline",
        "lastChangeTimestamp": "2022-07-31T08:50:37.9039274

Show predictions from reference set. Should not be drift or outliers.

In [11]:
batchSz=20
print(y_ref[0:batchSz])
infer("income-production.pipeline",batchSz,"normal")

[0 0 1 1 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 1]
<Response [200]>
{'model_name': '', 'outputs': [{'data': [0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1], 'name': 'predict', 'shape': [20], 'datatype': 'INT64'}, {'data': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'name': 'is_outlier', 'shape': [1, 20], 'datatype': 'INT64'}]}


In [13]:
!seldon pipeline inspect income-production.income-drift.outputs.is_drift

---
seldon.default.model.income-drift.outputs
cbj45dlvqj3madt364ug:{"name":"is_drift","datatype":"INT64","shape":["1"],"contents":{"int64Contents":["0"]}}


Show predictions from drift data. Should be drift and probably not outliers.

In [14]:
batchSz=20
print(y_ref[0:batchSz])
infer("income-production.pipeline",batchSz,"drift")

[0 0 1 1 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 1]
<Response [200]>
{'model_name': '', 'outputs': [{'data': [0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1], 'name': 'predict', 'shape': [20], 'datatype': 'INT64'}, {'data': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'name': 'is_outlier', 'shape': [1, 20], 'datatype': 'INT64'}]}


In [15]:
!seldon pipeline inspect income-production.income-drift.outputs.is_drift

---
seldon.default.model.income-drift.outputs
cbj45idvqj3madt364v0:{"name":"is_drift","datatype":"INT64","shape":["1"],"contents":{"int64Contents":["1"]}}


Show predictions from outlier data. Should be outliers and probably not drift.

In [16]:
batchSz=20
print(y_ref[0:batchSz])
infer("income-production.pipeline",batchSz,"outlier")

[0 0 1 1 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 1]
<Response [200]>
{'model_name': '', 'outputs': [{'data': [0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1], 'name': 'predict', 'shape': [20], 'datatype': 'INT64'}, {'data': [1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1], 'name': 'is_outlier', 'shape': [1, 20], 'datatype': 'INT64'}]}


In [17]:
!seldon pipeline inspect income-production.income-drift.outputs.is_drift

---
seldon.default.model.income-drift.outputs
cbj45ltvqj3madt364vg:{"name":"is_drift","datatype":"INT64","shape":["1"],"contents":{"int64Contents":["0"]}}


### Explanations

In [18]:
!cat ../../models/income-explainer.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: income-explainer
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/scv2/examples/income/explainer"
  explainer:
    type: anchor_tabular
    modelRef: income


In [19]:
!seldon model load -f ../../models/income-explainer.yaml

{}


In [20]:
!seldon model status income-explainer -w ModelAvailable | jq .

{}


In [21]:
batchSz=1
print(y_ref[0:batchSz])
infer("income-explainer",batchSz,"normal")

[0]
<Response [200]>
{'model_name': 'income-explainer_1', 'model_version': '1', 'id': '0400ed3e-498c-4703-bd61-75482129ea8b', 'parameters': {'content_type': None, 'headers': None}, 'outputs': [{'name': 'explanation', 'shape': [1], 'datatype': 'BYTES', 'parameters': {'content_type': 'str', 'headers': None}, 'data': ['{"meta": {"name": "AnchorTabular", "type": ["blackbox"], "explanations": ["local"], "params": {"seed": 1, "disc_perc": [25, 50, 75], "threshold": 0.95, "delta": 0.1, "tau": 0.15, "batch_size": 100, "coverage_samples": 10000, "beam_size": 1, "stop_on_first": false, "max_anchor_size": null, "min_samples_start": 100, "n_covered_ex": 10, "binary_cache_size": 10000, "cache_margin": 1000, "verbose": false, "verbose_every": 1, "kwargs": {}}, "version": "0.7.0"}, "data": {"anchor": ["Marital Status = Never-Married", "Relationship = Own-child", "Capital Gain <= 0.00"], "precision": 1.0, "coverage": 0.06853582554517133, "raw": {"feature": [3, 5, 8], "mean": [0.8037383177570093, 0.937

### Cleanup

In [22]:
!seldon model unload income-preprocess
!seldon model unload income
!seldon model unload income-drift
!seldon model unload income-outlier
!seldon model unload income-explainer

{}
{}
{}
{}
{}
